In [ ]:
#importing libraries
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join
import pywhatkit# WhatsApp and email
import datetime
import subprocess as sp
import time
import sys
#importing face classifier
face_classifier = cv2.CascadeClassifier(r'haarcascade_frontalface_default.xml')

In [ ]:
#function to extract images and convert them to gray scale
def face_extractor(img):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)    
    if faces is ():
        return None
    for (x,y,w,h) in faces:
        cropped_face = img[y:y+h, x:x+w]
    return cropped_face
cap = cv2.VideoCapture(0)
count = 0
while True:
    ret, frame = cap.read()
    if face_extractor(frame) is not None:
        count += 1
        face = cv2.resize(face_extractor(frame), (200, 200))
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        # Save file in specified directory with unique name
        file_name_path = 'C:/Users/faces/' + str(count) + '.jpg'
        cv2.imwrite(file_name_path, face)
        # Put count on images and display live count
        cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        cv2.imshow('Cropped Image', face)  
    else:
        print("Face not found")
        pass
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
cap.release()
cv2.destroyAllWindows()      
print("Image Collection Done !")
cap.release()

In [ ]:
model = cv2.face.LBPHFaceRecognizer_create()
data_path = 'C:/User/faces/'
onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]
Training_Data, Labels = [], []
for i, files in enumerate(onlyfiles):
    image_path = data_path + onlyfiles[i]
    images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    Training_Data.append(np.asarray(images, dtype=np.uint8))
    Labels.append(i)
Labels = np.asarray(Labels, dtype=np.int32)
model.train(np.asarray(Training_Data), np.asarray(Labels))
print("Model"+" trained sucessefully")

In [ ]:
face_classifier = cv2.CascadeClassifier(r'haarcascade_frontalface_default.xml')
def face_detector(img, size=0.5):
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5) 
    
    if faces is ():
        return img, []    
    
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
    return img, roi
# Open Webcam
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    image, face = face_detector(frame)
    try:
        face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
        results=model.predict(face)            
        if results[1]<500:
            confidence=int(100*(1-(results[1])/400))
            display=str(confidence)+'% Confident'
            cv2.putText(image, display, (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255))
            if confidence>90:
                cv2.show('Face recognised',image)
                pywhatkit.sendwhatmasq_instantly(phone_no="<phone_number>",message="Your face was recognised successfully!")
                pywhatkit.send_mail("sender_email","password","Face Recognition","User Recognised","receiver_email")
                break
            else:
                 os.system("aws ec2 run-instances --image-id ami-010aff33ed5991201 --instance-type t2.micro 
                          --subnet-id subnet-5793863f --count 1 --security-group-ids sg-00b5099795b410916 
                          --key-name key > ec2.txt")
                print("Instance launched !")
                os.system("aws ec2 create-volume --availability-zone ap-south-la --size 5 --volume-type group1 
                          --tag-specification Resourcetype=volume,Tags=[{Key=face, Value=vouleme}] > ebs.txt")
                print("Volume created , instance is initializing ......")
                time.sleep(120)
                ec2_id=open("ec2.txt",'r').read().split(',')[3].split(':')[1].split('""')[1]
                ebs_id=open("ebs.txt",'r').read().split(',')[6].split(':')[1].split('""')[1]         
                os.system("aws ec2 attach-volume --instance-id "+ec2_id+" --volume-id" +ebs_id+ " --device /dev/xvdf")
                print("Volume Attached to Instance")
                break                         

    except:
      cv2.putText(image, "No Face Found", (220, 120) , cv2.FONT_HERSHEY_COMPLEX, 1, (0,0,255), 2)
        cv2.imshow('Face Recognition', image )
        pass
        
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()